<h2 align='left' style='color:blue'>Dropout Regularization In Deep Neural Network</h2>

- This is a dataset that describes sonar chirp (noise) returns bouncing off different services. 
- The 60 input variables are the strength of the returns at different angles. 
- It is a **binary classification problem** that requires a model to differentiate rocks from metal cylinders.

- Dataset information: 
https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)
- Download it from here: 
https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [122]:
import warnings
warnings.filterwarnings('ignore')

In [123]:
#df = pd.read_csv("./sonar_dataset.csv")
# We don't have column name. Remove the column names with header = None.
df = pd.read_csv("./sonar_dataset.csv", header=None)   
df.sample(5)

0       1       2       3       4       5       6       7       8   \
181  0.0423  0.0321  0.0709  0.0108  0.1070  0.0973  0.0961  0.1323  0.2462   
106  0.0331  0.0423  0.0474  0.0818  0.0835  0.0756  0.0374  0.0961  0.0548   
165  0.0221  0.0065  0.0164  0.0487  0.0519  0.0849  0.0812  0.1833  0.2228   
19   0.0126  0.0149  0.0641  0.1732  0.2565  0.2559  0.2947  0.4110  0.4983   
53   0.0293  0.0378  0.0257  0.0062  0.0130  0.0612  0.0895  0.1107  0.0973   

         9   ...      51      52      53      54      55      56      57  \
181  0.2696  ...  0.0176  0.0035  0.0093  0.0121  0.0075  0.0056  0.0021   
106  0.0193  ...  0.0078  0.0174  0.0176  0.0038  0.0129  0.0066  0.0044   
165  0.1810  ...  0.0089  0.0051  0.0015  0.0075  0.0058  0.0016  0.0070   
19   0.5920  ...  0.0092  0.0035  0.0098  0.0121  0.0006  0.0181  0.0094   
53   0.0751  ...  0.0065  0.0072  0.0108  0.0051  0.0102  0.0041  0.0055   

         58      59  60  
181  0.0043  0.0017   M  
106  0.0134  0.0092   M  
165  0.0074  0.0038   M  
19   0.0116  0.0063   R  
53   0.0050  0.0087   R  

[5 rows x 61 columns]

In [124]:
df.shape

(208, 61)

In [125]:
# check for nan values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [126]:
df.columns

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [127]:
df[60].value_counts() # label is not skewed

M    111
R     97
Name: 60, dtype: int64

In [128]:
# X = df.drop(60, axis=1)
X = df.drop(60, axis="columns") # we drop columns 60.
y = df[60]
print('y.head():')
print(y.head())
X.head()

y.head():
0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object


0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      50      51      52      53      54      55      56  \
0  0.2111  ...  0.0232  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180   
1  0.2872  ...  0.0125  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140   
2  0.6194  ...  0.0033  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316   
3  0.1264  ...  0.0241  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050   
4  0.4459  ...  0.0156  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072   

       57      58      59  
0  0.0084  0.0090  0.0032  
1  0.0049  0.0052  0.0044  
2  0.0164  0.0095  0.0078  
3  0.0044  0.0040  0.0117  
4  0.0048  0.0107  0.0094  

[5 rows x 60 columns]

In [129]:
y = pd.get_dummies(y, drop_first=True) 
# after the dummpy is generate, 
# We drop the orginal variable R (Rock) or M (Mines) and keep the 1 and 0.
y.sample(5) # R --> 1 and M --> 0

R
62   1
11   1
89   1
149  0
142  0

In [130]:
y.value_counts()

R
0    111
1     97
dtype: int64

In [131]:
X.head()

0       1       2       3       4       5       6       7       8   \
0  0.0200  0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109   
1  0.0453  0.0523  0.0843  0.0689  0.1183  0.2583  0.2156  0.3481  0.3337   
2  0.0262  0.0582  0.1099  0.1083  0.0974  0.2280  0.2431  0.3771  0.5598   
3  0.0100  0.0171  0.0623  0.0205  0.0205  0.0368  0.1098  0.1276  0.0598   
4  0.0762  0.0666  0.0481  0.0394  0.0590  0.0649  0.1209  0.2467  0.3564   

       9   ...      50      51      52      53      54      55      56  \
0  0.2111  ...  0.0232  0.0027  0.0065  0.0159  0.0072  0.0167  0.0180   
1  0.2872  ...  0.0125  0.0084  0.0089  0.0048  0.0094  0.0191  0.0140   
2  0.6194  ...  0.0033  0.0232  0.0166  0.0095  0.0180  0.0244  0.0316   
3  0.1264  ...  0.0241  0.0121  0.0036  0.0150  0.0085  0.0073  0.0050   
4  0.4459  ...  0.0156  0.0031  0.0054  0.0105  0.0110  0.0015  0.0072   

       57      58      59  
0  0.0084  0.0090  0.0032  
1  0.0049  0.0052  0.0044  
2  0.0164  0.0095  0.0078  
3  0.0044  0.0040  0.0117  
4  0.0048  0.0107  0.0094  

[5 rows x 60 columns]

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [133]:
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_test.shape:', X_test.shape)
print('y_test.shape:', y_test.shape)

X_train.shape: (156, 60)
y_train.shape: (156, 1)
X_test.shape: (52, 60)
y_test.shape: (52, 1)


In [134]:
X_train.head()

0       1       2       3       4       5       6       7       8   \
67   0.0368  0.0403  0.0317  0.0293  0.0820  0.1342  0.1161  0.0663  0.0155   
14   0.0124  0.0433  0.0604  0.0449  0.0597  0.0355  0.0531  0.0343  0.1052   
164  0.0163  0.0198  0.0202  0.0386  0.0752  0.1444  0.1487  0.1484  0.2442   
179  0.0394  0.0420  0.0446  0.0551  0.0597  0.1416  0.0956  0.0802  0.1618   
19   0.0126  0.0149  0.0641  0.1732  0.2565  0.2559  0.2947  0.4110  0.4983   

         9   ...      50      51      52      53      54      55      56  \
67   0.0506  ...  0.0058  0.0091  0.0160  0.0160  0.0081  0.0070  0.0135   
14   0.2120  ...  0.0078  0.0083  0.0057  0.0174  0.0188  0.0054  0.0114   
164  0.2822  ...  0.0027  0.0077  0.0026  0.0031  0.0083  0.0020  0.0084   
179  0.2558  ...  0.0118  0.0146  0.0040  0.0114  0.0032  0.0062  0.0101   
19   0.5920  ...  0.0153  0.0092  0.0035  0.0098  0.0121  0.0006  0.0181   

         57      58      59  
67   0.0067  0.0078  0.0068  
14   0.0196  0.0147  0.0062  
164  0.0108  0.0083  0.0033  
179  0.0068  0.0053  0.0087  
19   0.0094  0.0116  0.0063  

[5 rows x 60 columns]

# Using Deep Learning Model

## Model without Dropout Layer

In [135]:
import tensorflow as tf
from tensorflow import keras

In [136]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 3ms/step - loss: 0.6904 - accuracy: 0.5064
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6724 - accuracy: 0.5769
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6526 - accuracy: 0.6154
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6238 - accuracy: 0.7308
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5903 - accuracy: 0.6923
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5627 - accuracy: 0.7372
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5206 - accuracy: 0.8013
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4898 - accuracy: 0.8462
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4759 - accuracy: 0.7821
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4777 - accuracy: 0.7821
Epoch 11/

In [137]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.8912 - accuracy: 0.7500


[0.8912267088890076, 0.75]

Training Accuracy >>> Test Accuracy

In [138]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[3.7654077e-06 9.7095126e-01 9.9986756e-01 2.6590304e-04 9.9999976e-01
 9.9974102e-01 1.9490720e-01 9.9999833e-01 3.9633218e-04 1.0000000e+00]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [139]:
y_test[:10]

R
186  0
155  0
165  0
200  0
58   1
34   1
151  0
18   1
202  0
62   1

In [140]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.85      0.78        27
           1       0.80      0.64      0.71        25

    accuracy                           0.75        52
   macro avg       0.76      0.75      0.75        52
weighted avg       0.76      0.75      0.75        52



### Model with Dropout Layer

In [141]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.6),  # drop 60%
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.6),
    keras.layers.Dense(1, activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 3ms/step - loss: 0.7199 - accuracy: 0.5385
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7354 - accuracy: 0.5128
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7348 - accuracy: 0.4936
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7499 - accuracy: 0.4615
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7132 - accuracy: 0.4615
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7300 - accuracy: 0.5000
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7005 - accuracy: 0.4808
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7291 - accuracy: 0.4808
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6871 - accuracy: 0.5449
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.7116 - accuracy: 0.4423
Epoch 11/

In [142]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.4985 - accuracy: 0.7692


[0.4984976649284363, 0.7692307829856873]

Training Accuracy is still good but Test Accuracy Improved

In [143]:
y_pred = modeld.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[0.00216362 0.59066105 0.67004037 0.05386209 0.6779506  0.6463295
 0.6392696  0.6631569  0.15134503 0.6712117 ]
[0. 1. 1. 0. 1. 1. 1. 1. 0. 1.]


In [144]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.81      0.79        27
           1       0.78      0.72      0.75        25

    accuracy                           0.77        52
   macro avg       0.77      0.77      0.77        52
weighted avg       0.77      0.77      0.77        52



**You can see that by using dropout layer test accuracy increased from 0.77 to 0.82**